In [ ]:
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);

SyntaxError: invalid syntax (<ipython-input-1-99b5b9a6ae1d>, line 1)

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import numpy as np
# import cv2
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils import to_categorical
# from tqdm import tqdm

# # 데이터를 불러오기
# data = np.load('/content/drive/MyDrive/지윤/npy/data.npy')
# target = np.load('/content/drive/MyDrive/지윤/npy/target.npy')

# num_classes = 10
# target_tmp = to_categorical(target)

# # 데이터셋 분할
# X_train, X_test, y_train, y_test = train_test_split(data, target_tmp, test_size=0.1, random_state=42)

# # 데이터셋 확인
# print("Training set size:", X_train.shape)
# print("Test set size:", X_test.shape)

In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

In [ ]:
model_try = str(10)

In [ ]:
IMGS = np.load('/content/drive/MyDrive/YANG/npy/img32_crop.npy')
RATIO = np.load('/content/drive/MyDrive/YANG/npy/ratios.npy')
TARGET = np.load('/content/drive/MyDrive/YANG/npy/target.npy')
TARGET_int = np.load('/content/drive/MyDrive/YANG/npy/target_int.npy')

print("IMGS shape:", IMGS.shape)
print("RATIO shape:", RATIO.shape)
print("TARGET shape:", TARGET.shape)
print("TARGET_int shape:", TARGET_int.shape)

IMGS shape: (18259, 224, 224, 3)
RATIO shape: (18259,)
TARGET shape: (18259, 5)
TARGET_int shape: (18259,)


In [ ]:
X_image_train, X_image_val, X_additional_train, X_additional_val, y_train, y_val = train_test_split(
    IMGS, RATIO, TARGET, test_size=0.2, random_state=42, stratify=TARGET_int)

In [ ]:
np.unique(TARGET_int)

array([1, 2, 3, 4, 5], dtype=int32)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 클래스 수 정의 (예시로 10을 사용)
num_classes = 5

# 이미지 입력 정의
image_input = Input(shape=(224, 224, 3))

# 사전 학습된 ResNet152V2 모델 로드 (최상위 레이어 제외)
base_model = ResNet152V2(weights='imagenet', include_top=False, input_tensor=image_input)

# GlobalAveragePooling2D 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)

# 추가 입력(상수) 정의
additional_input = Input(shape=(1,))

# 추가 입력을 기존 레이어와 결합
x = Concatenate()([x, additional_input])

# Fully Connected Layer 추가
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

# 모델 정의
model = Model(inputs=[image_input, additional_input], outputs=output)

# 일부 층은 훈련되지 않도록 설정 (동결)
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', # sgd, Nadam, Adagrad
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

234545216/234545216 [==============================] - 11s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                          